****Import required packages****

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

****Import Tensorflow****

In [ ]:
import tensorflow as tf
# required for solving issue with cuDNN
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

In [ ]:
tf.random.set_seed(42)

****Import Keras****

In [ ]:
from tensorflow import keras

In [ ]:
data_filename = '../data/ozone.csv'

In [ ]:
data = pd.read_csv(data_filename)

In [ ]:
data.shape

In [ ]:
data_columns = list(data.columns)

In [ ]:
print(data_columns)

In [ ]:
data.head()

In [ ]:
data.tail()

****Find number of records in each class****

In [ ]:
target_percentages = data['target'].value_counts()

In [ ]:
print(target_percentages)

In [ ]:
target_percentages = target_percentages.values

In [ ]:
target_0_percent = round(target_percentages[0] / np.sum(target_percentages) * 100, 3)

In [ ]:
print(target_0_percent)

In [ ]:
target_1_percent = round(target_percentages[1] / np.sum(target_percentages) * 100, 3)

In [ ]:
print(target_1_percent)

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [ ]:
data.isnull().any()

In [ ]:
for missing_flag in data.isnull().any():
    print(missing_flag)

In [ ]:
data == '?'

****Identify missing values with -1****

In [ ]:
data = data.replace('?', -1)

In [ ]:
data = data.iloc[:,1:].astype('float64')

In [ ]:
data.dtypes

In [ ]:
data.describe()

****Extract time series and targets from original data****

In [ ]:
timeseries = data.iloc[:,1:(data.shape[1] - 1)].values

In [ ]:
print(type(timeseries))

In [ ]:
timeseries.shape

In [ ]:
print(timeseries[0:5,-3:-1])

In [ ]:
targets_init = data['target'].values

In [ ]:
print(type(targets_init))

In [ ]:
targets_init.shape

In [ ]:
print(targets_init[0:5])

****Mean value imputation****

In [ ]:
def impute_data(timeseries):
    (n, d) = timeseries.shape
    timeseries_mean = np.mean(timeseries, axis=0)
    for i in range(n):
        for j in range(d):
            if timeseries[i, j] == -1:
                timeseries[i, j] = timeseries_mean[j]
    return timeseries

In [ ]:
timeseries = impute_data(timeseries)

In [ ]:
print(timeseries[0:5,-3:-1])

****Normalization****

In [ ]:
timeseries_mean = timeseries.mean(axis=0)

In [ ]:
print(timeseries_mean)

In [ ]:
timeseries_std = timeseries.std(axis=0)

In [ ]:
print(timeseries_std)

In [ ]:
timeseries -= timeseries_mean

In [ ]:
timeseries /= timeseries_std

In [ ]:
print(timeseries.mean(axis=0))

In [ ]:
print(timeseries.std(axis=0))

****Transform initial data into batch of sequences****

In [ ]:
def create_sequences(timeseries, targets_init, p):
    (n, d) = timeseries.shape
    sequences = np.zeros((n - p, p, d)).astype('float32')
    targets = np.zeros((n - p)).astype('float32')
    for i in range(p, n):
        sequence = timeseries[(i - p) : i, :]
        target = targets_init[i - 1]
        sequences[i - p, :, :] = sequence
        targets[i - p] = target
    return sequences, targets

In [ ]:
p = 5

In [ ]:
sequences, targets = create_sequences(timeseries, targets_init, p)

In [ ]:
sequences.shape

In [ ]:
targets.shape

****Split data****

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sequences, targets, test_size=0.1, shuffle=False)

In [ ]:
print(X_train.shape)

In [ ]:
print(y_train.shape)

In [ ]:
print(X_test.shape)

In [ ]:
print(y_test.shape)

****Target classes percentages in train data****

In [ ]:
unique_train, counts_train = np.unique(y_train, return_counts=True)

In [ ]:
print(dict(zip(unique_train, counts_train)))

In [ ]:
unique_train

In [ ]:
counts_train

In [ ]:
target_0_percent_train = round(counts_train[0] / np.sum(counts_train) * 100, 3)

In [ ]:
print(target_0_percent_train)

In [ ]:
target_1_percent_train = round(counts_train[1] / np.sum(counts_train) * 100, 3)

In [ ]:
print(target_1_percent_train)

****Target classes percentages in test data****

In [ ]:
unique_test, counts_test = np.unique(y_test, return_counts=True)

In [ ]:
print(dict(zip(unique_test, counts_test)))

In [ ]:
unique_test

In [ ]:
counts_test

In [ ]:
target_0_percent_test = round(counts_test[0] / np.sum(counts_test) * 100, 3)

In [ ]:
print(target_0_percent_test)

In [ ]:
target_1_percent_test = round(counts_test[1] / np.sum(counts_test) * 100, 3)

In [ ]:
print(target_1_percent_test)

****Function for plotting training history****

In [ ]:
def plot_training_history(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.figure()
    plt.plot(epochs, loss, label='Training loss')
    plt.plot(epochs, val_loss, label='Validation loss')
    plt.legend()
    plt.show()

****Baseline models****

****Persistence model****

In [ ]:
def baseline_predict(X):
    n = X.shape[0]
    preds = []
    for i in range(n):
        pred = 0 # always predict 0 class
        preds.append(pred)
    return np.array(preds)

In [ ]:
baseline_preds = baseline_predict(X_test)

In [ ]:
acc = keras.metrics.Accuracy()

In [ ]:
acc.update_state(y_true = y_test, y_pred = baseline_preds)

In [ ]:
round(acc.result().numpy() * 100, 3)

****Feedforward DNN****

In [ ]:
dnn = keras.Sequential()
dnn.add(keras.layers.Flatten(input_shape=(p, sequences.shape[2])))
dnn.add(keras.layers.Dense(16, activation='relu'))
dnn.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
dnn.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['acc'])

In [ ]:
history_dnn = dnn.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.1)

In [ ]:
dnn.summary()

In [ ]:
plot_training_history(history_dnn)

In [ ]:
dnn_results = dnn.evaluate(X_test, y_test)

In [ ]:
print('Feedforward DNN test accuracy (%): ', round(dnn_results[1] * 100, 3))

****Classic RNN****

In [ ]:
rnn = keras.Sequential()
rnn.add(keras.layers.SimpleRNN(16))
rnn.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
rnn.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['acc'])

In [ ]:
history_rnn = rnn.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.1)

In [ ]:
rnn.summary()

In [ ]:
plot_training_history(history_rnn)

In [ ]:
rnn_results = rnn.evaluate(X_test, y_test)

In [ ]:
print('Simple RNN test accuracy (%): ', round(rnn_results[1] * 100, 3))

****LSTM****

In [ ]:
lstm = keras.Sequential()
lstm.add(keras.layers.LSTM(16))
lstm.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
lstm.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['acc'])

In [ ]:
history_lstm = lstm.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.1)

In [ ]:
lstm.summary()

In [ ]:
plot_training_history(history_lstm)

In [ ]:
lstm_results = lstm.evaluate(X_test, y_test)

In [ ]:
print('LSTM test accuracy (%): ', round(lstm_results[1] * 100, 3))

****GRU****

In [ ]:
gru = keras.Sequential()
gru.add(keras.layers.GRU(16))
gru.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
gru.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['acc'])

In [ ]:
history_gru = gru.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.1)

In [ ]:
gru.summary()

In [ ]:
plot_training_history(history_gru)

In [ ]:
gru_results = gru.evaluate(X_test, y_test)

In [ ]:
print('GRU test accuracy (%): ', round(gru_results[1] * 100, 3))